In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
from temperature.exterior import *

In [3]:
temp = get_ext_df('temperature/export')

In [5]:
temp_int = pd.read_csv("temperature/netatmo_temperature_2021-01-01_2021-12-04.csv")
temp_int['date'] = pd.to_datetime(temp_int['date'])

In [7]:
px.line(temp_int, x='date', y='temperature')

In [26]:
gaz = pd.read_csv('gaz/gaz_conso.csv')

In [27]:
gaz['date'] = pd.to_datetime(gaz['date'], format="%d/%m/%Y")

In [28]:
fig = px.line(gaz, x='date', y='energy')
fig.show()

In [29]:
# compare with last year for a given month

def compare_last_year(df, month, number_days_month):
    df_month = df[df.date.dt.month == month].sort_values(by='date')
    df_month['energy_last_year'] = df_month['energy'].shift(number_days_month)
    df_month['diff'] = df_month['energy'] - df_month['energy_last_year']
    return df_month



In [30]:
temp['datetime'] = pd.to_datetime(temp['datetime'], format="%Y-%m-%d %H:%M")

In [31]:
temp['date'] = temp.datetime.dt.date

In [32]:
temp_df = temp.groupby('date').mean().reset_index()
temp_df['date'] = pd.to_datetime(temp_df['date'], format="%Y-%m-%d")

In [33]:
df_merge = gaz.merge(temp_df, on='date', how='left')

In [34]:
df_merge.dropna(inplace=True)

In [35]:
px.line(df_merge, x='date', y='temp').show()

In [36]:
def add_dim_installation(x):
    if x > datetime.strptime("2021-07-17", "%Y-%m-%d"):
        return 1
    else: 
        return 0

In [37]:
df_merge['dimension'] = df_merge['date'].apply(add_dim_installation)
gaz['dimension'] = gaz['date'].apply(add_dim_installation)

In [41]:
px.scatter(df_merge, x='temp', y='energy', trendline="ols")

In [44]:
scatter = px.scatter(df_merge, x='temp', y='energy', color="dimension")
scatter.show()

In [45]:
def run_regression(df, x, y):
    X = df[x]
    Y = df[y]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    return model.summary()

In [46]:
df_merge[df_merge.date.dt.month.isin([10,11])].groupby('dimension').mean()

,volume,energy,temp
dimension,,,
0,3.050847,34.576271,11.266132
1,2.214286,25.267857,10.517003


In [52]:
df_merge['year_month'] = df_merge['date'].dt.strftime('%Y-%m')
df_merge.groupby('year_month').mean().sort_values(by='temp')

,volume,energy,temp,dimension
year_month,,,,
2021-01,7.806452,87.774194,5.067602,0.000000
2020-12,6.258065,69.677419,6.977560,0.000000
2021-02,5.107143,59.642857,6.989130,0.000000
2021-11,3.640000,40.760000,7.864870,1.000000
2021-03,5.516129,61.451613,8.965498,0.000000
2020-11,4.535714,50.750000,9.808139,0.000000
2021-04,2.433333,27.833333,10.154203,0.000000
2020-10,1.709677,19.967742,12.583029,0.000000
2021-10,1.064516,12.774194,12.655820,1.000000


In [48]:
run_regression(df_merge[df_merge.date.dt.month.isin([11])], ['temp', 'dimension'], ['energy'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 energy   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     56.99
Date:                Tue, 30 Nov 2021   Prob (F-statistic):           1.27e-13
Time:                        21:41:19   Log-Likelihood:                -202.44
No. Observations:                  53   AIC:                             410.9
Df Residuals:                      50   BIC:                             416.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        108.8454      6.094     17.862      0.000      96.606     121.085
temp          -5.9232      0.581    -10.186      0.000      -7.091      -4.755
dimension    -21.5003      3.323     -6.471      0.000     -28.174     -14.826
==============================================================================
Omnibus:                       22.121   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.443
Skew:                          -1.370   Prob(JB):                     2.01e-08
Kurtosis:                       5.923   Cond. No.                         38.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""